In [ ]:

#Imports
# =========================
import json
import time
import numpy as np
import joblib
import optuna
import mlflow
from pathlib import Path
import sqlite3
import pandas as pd

REPO_DIR = Path.cwd().parent if Path.cwd().name == "notebooks" else Path.cwd()
DATA_DIR = REPO_DIR / "data"
DATA_DIR.mkdir(exist_ok=True)

DB_PATH = DATA_DIR / "classification.db"

print("✅ DB_PATH:", DB_PATH)




from pathlib import Path
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.metrics import f1_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split


from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import HistGradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier


# =========================
#Paths
# =========================
REPO_DIR = Path.cwd().parent if Path.cwd().name == "notebooks" else Path.cwd()
DATA_DIR = REPO_DIR / "data"
MODELS_DIR = REPO_DIR / "models"
METRICS_DIR = REPO_DIR / "metrics"

MODELS_DIR.mkdir(exist_ok=True)
METRICS_DIR.mkdir(exist_ok=True)


# =========================
#Preprocessing
# =========================
preprocess = Pipeline([
    ("scaler", StandardScaler())
])


# =========================
#Model factory
# =========================
def make_model(model_name: str):
    name = model_name.lower().strip()

    if name in ["ridge", "logreg", "logistic"]:
        return LogisticRegression(
            max_iter=2000,
            penalty="l2",
            solver="lbfgs"
        )

    if name in ["histgradientboosting", "hgb"]:
        return HistGradientBoostingClassifier(random_state=42)

    if name in ["xgboost", "xgb"]:
        return XGBClassifier(
            random_state=42,
            eval_metric="logloss",
            use_label_encoder=False
        )

    if name in ["lightgbm", "lgbm"]:
        return LGBMClassifier(random_state=42)

    raise ValueError(f"Unknown model_name: {model_name}")


# =========================
#Pipeline builder
# =========================
def build_pipeline(model_name: str, use_pca: bool, tuned_params=None):
    model = make_model(model_name)

    if tuned_params:
        model.set_params(**tuned_params)

    steps = [("preprocess", preprocess)]

    if use_pca:
        steps.append(("pca", PCA(n_components=0.95, random_state=42)))

    steps.append(("model", model))

    return Pipeline(steps)


# =========================
#Hyperparameter suggestions
# =========================
def suggest_params(trial, model_name):
    name = model_name.lower()

    if name in ["ridge", "logreg"]:
        return {
            "model__C": trial.suggest_float("C", 0.01, 10.0, log=True)
        }

    if name in ["xgboost", "xgb"]:
        return {
            "model__n_estimators": trial.suggest_int("n_estimators", 100, 500),
            "model__max_depth": trial.suggest_int("max_depth", 3, 8)
        }

    if name in ["lightgbm", "lgbm"]:
        return {
            "model__n_estimators": trial.suggest_int("n_estimators", 100, 500)
        }

    if name in ["histgradientboosting", "hgb"]:
        return {
            "model__max_depth": trial.suggest_int("max_depth", 3, 10)
        }

    return {}


# =========================
#Single experiment runner
# =========================
def run_one_experiment(model_name, use_pca, use_optuna, n_trials=20):
    run_name = f"{model_name}__{'pca' if use_pca else 'no_pca'}__{'optuna' if use_optuna else 'no_optuna'}"
    print("\n" + "="*80)
    print("RUN:", run_name)
    print("="*80)

    cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

    best_params = None

    #Optuna 
    if use_optuna:
        def objective(trial):
            params = suggest_params(trial, model_name)
            pipe = build_pipeline(model_name, use_pca, params)
            scores = cross_val_score(pipe, X_train, y_train, cv=cv, scoring="f1", n_jobs=-1)
            return float(np.mean(scores))

        study = optuna.create_study(direction="maximize")
        study.optimize(objective, n_trials=n_trials, show_progress_bar=False)

        best_params = study.best_params
        best_cv_f1 = float(study.best_value)

    else:
        pipe_tmp = build_pipeline(model_name, use_pca, None)
        scores = cross_val_score(pipe_tmp, X_train, y_train, cv=cv, scoring="f1", n_jobs=-1)
        best_cv_f1 = float(np.mean(scores))

    #Train final
    pipe = build_pipeline(model_name, use_pca, best_params)
    pipe.fit(X_train, y_train)
    y_pred = pipe.predict(X_test)
    test_f1 = float(f1_score(y_test, y_pred))

    print("cv_f1 :", round(best_cv_f1, 4))
    print("test_f1:", round(test_f1, 4))

    model_path = MODELS_DIR / f"{run_name}.joblib"
    metrics_path = METRICS_DIR / f"{run_name}.json"

    joblib.dump(pipe, model_path)

    payload = {
        "run_name": run_name,
        "model_family": model_name,
        "uses_pca": use_pca,
        "uses_optuna": use_optuna,
        "cv_f1": best_cv_f1,
        "test_f1": test_f1,
        "best_params": best_params,
        "model_path": str(model_path),
        "metrics_path": str(metrics_path),
    }

    with open(metrics_path, "w", encoding="utf-8") as f:
        json.dump(payload, f, indent=2)

    #Log to MLflow (Dagshub)
    with mlflow.start_run(run_name=run_name):
        mlflow.log_param("model_family", model_name)
        mlflow.log_param("uses_pca", use_pca)
        mlflow.log_param("uses_optuna", use_optuna)

        if best_params:
            mlflow.log_params(best_params)

        mlflow.log_metric("cv_f1", best_cv_f1)
        mlflow.log_metric("test_f1", test_f1)

        mlflow.log_artifact(str(model_path), artifact_path="models")
        mlflow.log_artifact(str(metrics_path), artifact_path="metrics")

    return payload


✅ DB_PATH: c:\Users\preon\OneDrive\Desktop\final project\sql-to-ml-pipeline\data\classification.db


In [12]:
# =============================================================================
# FULL PIPELINE:
# - Build preprocessing
# - Stratified train/test split
# - Train & log 4 models WITHOUT PCA (Ridge, HGB, XGBoost, LightGBM)
# - Train & log 4 models WITH PCA (preprocessing + PCA(0.95) + model)
# - Pick GLOBAL best among 8 models by Test MAE
# - Save, load, and compare the global best model
# =============================================================================


conn = sqlite3.connect(str(DB_PATH))

df = pd.read_sql("""
    SELECT f.*, l.label
    FROM features f
    JOIN labels l ON f.row_id = l.row_id
""", conn)

conn.close()

print("✅ Joined df:", df.shape)
print(df["label"].value_counts())
df.head()



✅ Joined df: (569, 32)
label
1    357
0    212
Name: count, dtype: int64


,row_id,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,label
0,1,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0
1,2,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0
2,3,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0
3,4,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,0
4,5,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,0


In [13]:
TARGET_COL = "label"
ID_COL = "row_id"

X = df.drop(columns=[TARGET_COL, ID_COL])
y = df[TARGET_COL]

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.20,
    random_state=42,
    stratify=y
)

print("✅ Split done:")
print("Train:", X_train.shape, "Test:", X_test.shape)
print("Train label dist:\n", y_train.value_counts(normalize=True))
print("Test  label dist:\n", y_test.value_counts(normalize=True))


✅ Split done:
Train: (455, 30) Test: (114, 30)
Train label dist:
 label
1    0.626374
0    0.373626
Name: proportion, dtype: float64
Test  label dist:
 label
1    0.631579
0    0.368421
Name: proportion, dtype: float64


In [14]:
preprocess = Pipeline(steps=[
    ("scaler", StandardScaler())
])

def make_model(model_name: str):
    name = model_name.lower().strip()

    if name in ["logreg", "logistic"]:
        return LogisticRegression(
            max_iter=2000,
            solver="lbfgs"
        )

    if name in ["histgradientboosting", "hgb"]:
        return HistGradientBoostingClassifier(random_state=42)

    if name in ["xgboost", "xgb"]:
        return XGBClassifier(
            random_state=42,
            eval_metric="logloss",
            use_label_encoder=False
        )

    if name in ["lightgbm", "lgbm"]:
        return LGBMClassifier(random_state=42)

    raise ValueError(f"Unknown model_name: {model_name}")


In [15]:
def suggest_params(trial, model_name):
    name = model_name.lower()

    if name in ["logreg", "logistic"]:
        return {
            "model__C": trial.suggest_float("C", 0.01, 10.0, log=True)
        }

    if name in ["xgboost", "xgb"]:
        return {
            "model__n_estimators": trial.suggest_int("n_estimators", 100, 500),
            "model__max_depth": trial.suggest_int("max_depth", 3, 8)
        }

    if name in ["lightgbm", "lgbm"]:
        return {
            "model__n_estimators": trial.suggest_int("n_estimators", 100, 500)
        }

    if name in ["histgradientboosting", "hgb"]:
        return {
            "model__max_depth": trial.suggest_int("max_depth", 3, 10)
        }

    return {}



In [ ]:
# =========================
# Imports (safe to repeat)
# =========================
from pathlib import Path
import json
import numpy as np

import joblib
import optuna
import mlflow
import dagshub

from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.metrics import f1_score


# =========================
# Paths
# =========================
REPO_DIR = Path.cwd().parent if Path.cwd().name == "notebooks" else Path.cwd()
MODELS_DIR = REPO_DIR / "models"
METRICS_DIR = REPO_DIR / "metrics"
MODELS_DIR.mkdir(exist_ok=True)
METRICS_DIR.mkdir(exist_ok=True)


# =========================
# Dagshub MLflow init
# =========================
dagshub.init(repo_owner="RamishaPrionti", repo_name="sql-to-ml-pipeline", mlflow=True)
mlflow.set_experiment("sql_to_ml_pipeline")


# =========================
# REQUIRED: preprocess + model factory

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import HistGradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

def make_model(model_name: str):
    name = model_name.lower()
    if name in ["logreg", "logistic", "logistic_regression"]:
        return LogisticRegression(max_iter=2000)
    if name in ["histgradientboosting", "hgb", "hist_gb"]:
        return HistGradientBoostingClassifier(random_state=42)
    if name in ["xgboost", "xgb"]:
        return XGBClassifier(
            random_state=42,
            eval_metric="logloss",
            use_label_encoder=False
        )
    if name in ["lightgbm", "lgbm"]:
        return LGBMClassifier(random_state=42)
    raise ValueError(f"Unknown model_name: {model_name}")


def suggest_params(trial, model_name: str):
    name = model_name.lower()

    if name in ["logreg", "logistic", "logistic_regression"]:
        return {
            "C": trial.suggest_float("C", 1e-3, 10.0, log=True),
            "solver": trial.suggest_categorical("solver", ["liblinear", "lbfgs"]),
        }

    if name in ["histgradientboosting", "hgb", "hist_gb"]:
        return {
            "max_depth": trial.suggest_int("max_depth", 2, 10),
            "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
            "max_iter": trial.suggest_int("max_iter", 100, 400),
        }

    if name in ["xgboost", "xgb"]:
        return {
            "n_estimators": trial.suggest_int("n_estimators", 100, 500),
            "max_depth": trial.suggest_int("max_depth", 2, 8),
            "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
            "subsample": trial.suggest_float("subsample", 0.6, 1.0),
            "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 1.0),
        }

    if name in ["lightgbm", "lgbm"]:
        return {
            "n_estimators": trial.suggest_int("n_estimators", 100, 500),
            "max_depth": trial.suggest_int("max_depth", -1, 12),
            "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
            "num_leaves": trial.suggest_int("num_leaves", 16, 128),
        }

    raise ValueError(f"No suggest_params configured for: {model_name}")



def build_pipeline(model_name: str, use_pca: bool, tuned_params: dict | None):
    est = make_model(model_name)
    if tuned_params:
        est.set_params(**tuned_params)

    steps = [("preprocess", preprocess)]
    if use_pca:
        steps.append(("pca", PCA(n_components=0.95, random_state=42)))
    steps.append(("model", est))
    return Pipeline(steps)


def run_one_experiment(model_name: str, use_pca: bool, use_optuna: bool, n_trials: int = 20):
    run_name = f"{model_name}__{'pca' if use_pca else 'no_pca'}__{'optuna' if use_optuna else 'no_optuna'}"
    print("\n" + "=" * 80)
    print("RUN:", run_name)
    print("=" * 80)

    cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

    best_params = None

    #Optuna tuning-
    if use_optuna:
        def objective(trial):
            params = suggest_params(trial, model_name)
            pipe = build_pipeline(model_name, use_pca, params)
            scores = cross_val_score(pipe, X_train, y_train, cv=cv, scoring="f1", n_jobs=-1)
            return float(np.mean(scores))

        study = optuna.create_study(direction="maximize")
        study.optimize(objective, n_trials=n_trials, show_progress_bar=False)

        best_params = study.best_params
        best_cv_f1 = float(study.best_value)

    #No optuna:
    else:
        pipe_tmp = build_pipeline(model_name, use_pca, tuned_params=None)
        scores = cross_val_score(pipe_tmp, X_train, y_train, cv=cv, scoring="f1", n_jobs=-1)
        best_cv_f1 = float(np.mean(scores))

   
    pipe = build_pipeline(model_name, use_pca, best_params)
    pipe.fit(X_train, y_train)
    y_pred = pipe.predict(X_test)
    test_f1 = float(f1_score(y_test, y_pred))

    print("cv_f1 :", round(best_cv_f1, 4))
    print("test_f1:", round(test_f1, 4))
    if best_params:
        print("best_params:", best_params)

    #Save locally 
    model_path = MODELS_DIR / f"{run_name}.joblib"
    metrics_path = METRICS_DIR / f"{run_name}.json"
    joblib.dump(pipe, model_path)

    payload = {
        "run_name": run_name,
        "model_family": model_name,
        "uses_pca": use_pca,
        "uses_optuna": use_optuna,
        "cv_f1": best_cv_f1,
        "test_f1": test_f1,
        "best_params": best_params,
        "model_path": str(model_path),
        "metrics_path": str(metrics_path),
    }

    with open(metrics_path, "w", encoding="utf-8") as f:
        json.dump(payload, f, indent=2)

    
    with mlflow.start_run(run_name=run_name):
        mlflow.log_param("model_family", model_name)
        mlflow.log_param("uses_pca", use_pca)
        mlflow.log_param("uses_optuna", use_optuna)
        if best_params:
            mlflow.log_params(best_params)

        mlflow.log_metric("cv_f1", best_cv_f1)
        mlflow.log_metric("test_f1", test_f1)

        mlflow.log_artifact(str(model_path), artifact_path="models")
        mlflow.log_artifact(str(metrics_path), artifact_path="metrics")

    return payload 


def run_all_16():
    model_list = ["logreg", "histgradientboosting", "xgboost", "lightgbm"]
    all_payloads = []
    for model_name in model_list:
        for use_pca in [False, True]:
            for use_optuna in [False, True]:
                all_payloads.append(run_one_experiment(model_name, use_pca, use_optuna, n_trials=20))
    return all_payloads

    

Accessing as RamishaPrionti

Initialized MLflow to track repo "RamishaPrionti/sql-to-ml-pipeline"

Repository RamishaPrionti/sql-to-ml-pipeline initialized!

In [17]:
MODEL_NAMES = ["logreg", "hgb", "xgb", "lgbm"]
all_runs = []
start = time.time()

for model_name in MODEL_NAMES:
    all_runs.append(run_one_experiment(model_name, use_pca=False, use_optuna=False))
    all_runs.append(run_one_experiment(model_name, use_pca=True,  use_optuna=False))
    all_runs.append(run_one_experiment(model_name, use_pca=False, use_optuna=True, n_trials=20))
    all_runs.append(run_one_experiment(model_name, use_pca=True,  use_optuna=True, n_trials=20))

elapsed = time.time() - start
print("\n✅ DONE. Total runs:", len(all_runs))
print("Elapsed minutes:", round(elapsed/60, 2))



RUN: logreg__no_pca__no_optuna
cv_f1 : 0.9808
test_f1: 0.9861
🏃 View run logreg__no_pca__no_optuna at: https://dagshub.com/RamishaPrionti/sql-to-ml-pipeline.mlflow/#/experiments/0/runs/28efeb3a8b0546bfab2e579bfed7d242
🧪 View experiment at: https://dagshub.com/RamishaPrionti/sql-to-ml-pipeline.mlflow/#/experiments/0

RUN: logreg__pca__no_optuna
cv_f1 : 0.9861
test_f1: 0.979
🏃 View run logreg__pca__no_optuna at: https://dagshub.com/RamishaPrionti/sql-to-ml-pipeline.mlflow/#/experiments/0/runs/f326a0a5b315437b8781be4bcd70c959
🧪 View experiment at: https://dagshub.com/RamishaPrionti/sql-to-ml-pipeline.mlflow/#/experiments/0


[I 2025-12-18 12:30:27,376] A new study created in memory with name: no-name-04bad6be-7179-441e-865e-ff9651289441



RUN: logreg__no_pca__optuna


[I 2025-12-18 12:30:29,162] Trial 0 finished with value: 0.9790558722655804 and parameters: {'C': 1.7479665886752973, 'solver': 'lbfgs'}. Best is trial 0 with value: 0.9790558722655804.
[I 2025-12-18 12:30:30,832] Trial 1 finished with value: 0.9665244700662009 and parameters: {'C': 0.0034499339772257496, 'solver': 'liblinear'}. Best is trial 0 with value: 0.9790558722655804.
[I 2025-12-18 12:30:32,636] Trial 2 finished with value: 0.9755471003357559 and parameters: {'C': 3.813275454197846, 'solver': 'liblinear'}. Best is trial 0 with value: 0.9790558722655804.
[I 2025-12-18 12:30:34,399] Trial 3 finished with value: 0.9794508439114478 and parameters: {'C': 0.0394443224261552, 'solver': 'lbfgs'}. Best is trial 3 with value: 0.9794508439114478.
[I 2025-12-18 12:30:36,087] Trial 4 finished with value: 0.9755471003357559 and parameters: {'C': 2.394260443576521, 'solver': 'liblinear'}. Best is trial 3 with value: 0.9794508439114478.
[I 2025-12-18 12:30:36,112] Trial 5 finished with value: 

cv_f1 : 0.9827
test_f1: 0.9793
best_params: {'C': 0.12822475443271025, 'solver': 'lbfgs'}
🏃 View run logreg__no_pca__optuna at: https://dagshub.com/RamishaPrionti/sql-to-ml-pipeline.mlflow/#/experiments/0/runs/aa7d15d64cb944f1ba764c095f79c4d5
🧪 View experiment at: https://dagshub.com/RamishaPrionti/sql-to-ml-pipeline.mlflow/#/experiments/0


...

[LightGBM] [Info] Number of positive: 285, number of negative: 170
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000419 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4545
[LightGBM] [Info] Number of data points in the train set: 455, number of used features: 30
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.626374 -> initscore=0.516691
[LightGBM] [Info] Start training from score 0.516691
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best 

[I 2025-12-18 12:33:00,985] A new study created in memory with name: no-name-718472f9-eb9f-4a97-9e86-60d8fa27cc99


🏃 View run lgbm__no_pca__optuna at: https://dagshub.com/RamishaPrionti/sql-to-ml-pipeline.mlflow/#/experiments/0/runs/d13f835d8b97423eb6896ce45ce651df
🧪 View experiment at: https://dagshub.com/RamishaPrionti/sql-to-ml-pipeline.mlflow/#/experiments/0

RUN: lgbm__pca__optuna


[I 2025-12-18 12:33:01,064] Trial 0 finished with value: 0.9599714008361504 and parameters: {'n_estimators': 184, 'max_depth': 2, 'learning_rate': 0.26357932650032845, 'num_leaves': 23}. Best is trial 0 with value: 0.9599714008361504.
[I 2025-12-18 12:33:01,256] Trial 1 finished with value: 0.9597071405823198 and parameters: {'n_estimators': 166, 'max_depth': 6, 'learning_rate': 0.026204883481624665, 'num_leaves': 41}. Best is trial 0 with value: 0.9599714008361504.
[I 2025-12-18 12:33:01,572] Trial 2 finished with value: 0.95453216374269 and parameters: {'n_estimators': 269, 'max_depth': 9, 'learning_rate': 0.012089509782912766, 'num_leaves': 54}. Best is trial 0 with value: 0.9599714008361504.
[I 2025-12-18 12:33:02,004] Trial 3 finished with value: 0.9653292678558637 and parameters: {'n_estimators': 327, 'max_depth': 8, 'learning_rate': 0.03918882962106877, 'num_leaves': 81}. Best is trial 3 with value: 0.9653292678558637.
[I 2025-12-18 12:33:02,155] Trial 4 finished with value: 0.9

[LightGBM] [Info] Number of positive: 285, number of negative: 170
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000331 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1520
[LightGBM] [Info] Number of data points in the train set: 455, number of used features: 10
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.626374 -> initscore=0.516691
[LightGBM] [Info] Start training from score 0.516691
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best 

In [21]:
best = results_df.iloc[0].to_dict()

print("✅ BEST RUN:", best["run_name"])
print("Best test_f1:", best["test_f1"])

best_model = joblib.load(best["model_path"])

FINAL_MODEL_PATH = MODELS_DIR / "final_model.joblib"
joblib.dump(best_model, FINAL_MODEL_PATH)

print("✅ Saved FINAL model to:", FINAL_MODEL_PATH)



✅ BEST RUN: lgbm__pca__optuna
Best test_f1: 0.9863013698630136
✅ Saved FINAL model to: c:\Users\preon\OneDrive\Desktop\final project\sql-to-ml-pipeline\models\final_model.joblib
